# タスク 1d: テキスト生成の実行 (マルチモーダル)


### シナリオ

## タスク 1c.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [11]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## タスク 1c.2: テキストの生成

このタスクでは、Amazon Bedrock サービスがメールを生成するための入力を準備します。

In [12]:
# create the prompt
prompt_data = """
添付した画像ファイルは AWS のアーキテクチャ図です。
この構成を実現する CloudFormation テンプレートを作成してください
"""

次に、Amazon Nova Lite を呼び出します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** Amazon Nova Lite は、約 300kトークンのコンテキストウィンドウをサポートし、次のパラメータを受け入れます。:
- `messages`: ロールとコンテンツを持つメッセージオブジェクトの配列
- `inferenceConfig`: これらは、モデルが出力を生成する際に考慮するパラメータです。

Amazon Bedrock API は、次のものを受け入れる API `invoke_model` を提供します:
- `modelId`: これは、Amazon Bedrock で使用できるさまざまな基盤モデルのモデル ARN です
- `accept`: 入力リクエストのタイプ
- `contentType`: 出力のコンテンツ タイプ
- `body`: プロンプトと構成で構成される JSON 文字列

使用可能なテキスト生成モデル ID については、[ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) を参照してください。

## タスク 1a.3: Amazon Nova Lite 言語モデルを呼び出す

このタスクでは、以前に作成したプロンプトに基づいてモデルが出力を生成する方法を調べます。

### 完全な出力生成

このメールは、入力リクエストを理解し、さまざまなモダリティに対する固有の理解を活用して、Amazon Nova Lite モデルを使用して生成されます。この　API　は同期リクエストのため、モデルによって出力全体が生成されるまで待機します。

In [13]:
with open('Amazon-Bedrock.png', 'rb') as f:
    png_bytes = f.read()

In [14]:
#invoke model
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
outputText = "\n"
try:

    response = bedrock_client.converse(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "text": prompt_data
                    }, {
                        # "text": prompt_data,
                        "image": {
                            "format": "png",
                            "source": {
                                "bytes": png_bytes
                                # 's3Location': {
                                #     'uri': 'string',
                                #     'bucketOwner': 'string'
                                # }
                            }
                        }
                    }
                ]
            }
        ],
        modelId=modelId,
        inferenceConfig = {
            "maxTokens": 8192,
            "stopSequences": [],
            "temperature": 0,
            "topP": 0.9
        }
    )
    outputText = response.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [15]:
# Print the complete response from Nova Lite
print(outputText)


Below is a basic AWS CloudFormation template based on the architecture diagram you provided. This template assumes you have a basic understanding of AWS resources and CloudFormation syntax.

```yaml
AWSTemplateFormatVersion: '2010-09-09'
Description: Infrastructure as Code for the provided architecture

Resources:
  # VPC
  MyVPC:
    Type: 'AWS::EC2::VPC'
    Properties:
      CidrBlock: '10.0.0.0/16'

  # Subnets
  PublicSubnet1:
    Type: 'AWS::EC2::Subnet'
    Properties:
      VpcId: !Ref MyVPC
      CidrBlock: '10.0.1.0/24'
      MapPublicIpOnLaunch: true

  # Security Group for Amazon EC2 Fargate
  FargateSecurityGroup:
    Type: 'AWS::EC2::SecurityGroup'
    Properties:
      GroupDescription: 'Allow access to Fargate'
      VpcId: !Ref MyVPC
      SecurityGroupIngress:
        - IpProtocol: 'tcp'
          FromPort: '80'
          ToPort: '80'
          CidrIp: '0.0.0.0/0'

  # Fargate Cluster
  FargateCluster:
    Type: 'AWS::ECS::Cluster'

  # Fargate Task Definition
  Farga


これで、Amazon Bedrock API を使用するためのの基本的な boto3 SDK を試しました。この API を使用して、顧客の否定的なフィードバックに応答するメールを生成するユースケースを確認しました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、 **Task1b_ja_jp.ipynb** を開きます。